In [3]:
Pkg.update()

INFO: Updating METADATA...
INFO: Computing changes...
INFO: No packages to install, update or remove


In [4]:
using DataFrames, DistributedArrays, Images, Colors, FixedPointNumbers, ImageView, Grid, DecisionTree, JLD, HDF5

    .+(Images.AbstractImageDirect, AbstractArray) at C:\Users\cyber\.julia\v0.4\Images\src\algorithms.jl:22
is ambiguous with: 
    .+(AbstractArray, Union{DataArrays.PooledDataArray, DataArrays.DataArray}, AbstractArray...) at C:\Users\cyber\.julia\v0.4\DataArrays\src\broadcast.jl:297.
To fix, define 
    .+(Images.AbstractImageDirect, Union{DataArrays.PooledDataArray, DataArrays.DataArray})
before the new definition.
    +(AbstractArray, Images.AbstractImageDirect) at C:\Users\cyber\.julia\v0.4\Images\src\algorithms.jl:20
is ambiguous with: 
    +(DataArrays.DataArray, AbstractArray) at C:\Users\cyber\.julia\v0.4\DataArrays\src\operators.jl:326.
To fix, define 
    +(DataArrays.DataArray, Images.AbstractImageDirect)
before the new definition.
    +(AbstractArray, Images.AbstractImageDirect) at C:\Users\cyber\.julia\v0.4\Images\src\algorithms.jl:20
is ambiguous with: 
    +(DataArrays.AbstractDataArray, AbstractArray) at C:\Users\cyber\.julia\v0.4\DataArrays\src\operators.jl:349.
To f

In [5]:
# generic function to read from train and test images
function read_data(typeData, labelsInfo, width, height, path)
    #Intialize x matrix
    x = zeros(size(labelsInfo, 1), width * height)
    
    last = size(labelsInfo)[1]
    for i = 1:last
        # Since train images are arranged into folders, we need to loop through all C0 thru C9 of them
        if typeData == "train"
            class = labelsInfo[:classname][i]
            image = labelsInfo[:img][i]
            nameFile = "$(path)/$(typeData)/$(class)/$(image)"
        else
            image = labelsInfo[:img][i]
            nameFile = "$(path)/$(typeData)/$(image)"
        end
        #println("Reading Image ==> $(nameFile)")        
        
        # Load the image file
        #img = imread(nameFile)
        img = load(nameFile)

        #Convert the image to GrayScale
        img_gsc = convert(Image{Gray}, img)
        
        # Resize the image
        img_gsc_rsz = Images.imresize(img_gsc, (width, height))
        
        # Convert the image into an array of floats
        temp = float32(img_gsc_rsz)
        
        #Transform image matrix to a vector and store it
        x[i, :] = vec(temp)
    end 
    return x
end

read_data (generic function with 1 method)

In [6]:
# read the train and test labels from the current folder
path = "."

#Read information about test data ( IDs ).
labelsInfoTest = readtable("$(path)/sample_submission.csv")

#Read information about training data , IDs.
labelsInfoTrain = readtable("$(path)/driver_imgs_list.csv")
println("")

In [7]:
head(labelsInfoTrain)

,subject,classname,img
1,p002,c0,img_44733.jpg
2,p002,c0,img_72999.jpg
3,p002,c0,img_25094.jpg
4,p002,c0,img_69092.jpg
5,p002,c0,img_92629.jpg
6,p002,c0,img_3370.jpg


In [8]:
head(labelsInfoTest)

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
1,img_1.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2,img_10.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,img_100.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
4,img_1000.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
5,img_100000.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
6,img_100001.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [9]:
# navigate to imgs folder which contains train and test data folders
path = "./imgs"

# set the dimensions to which the image needs to be scaled (the original images are 640 X 480)
width = 160
height = 120

#Read training matrix
xTrain = read_data("train", labelsInfoTrain, width, height, path)

22424x19200 Array{Float64,2}:
 0.105882   0.113725   0.101961   …  0.0588235  0.101961   0.180392 
 0.0862745  0.0745098  0.0823529     0.0235294  0.0784314  0.133333 
 0.0627451  0.0627451  0.0745098     0.0509804  0.0784314  0.12549  
 0.0509804  0.0627451  0.054902      0.0352941  0.0392157  0.0392157
 0.0588235  0.0588235  0.0588235     0.0235294  0.0352941  0.054902 
 0.0745098  0.0352941  0.0509804  …  0.027451   0.0509804  0.0705882
 0.0470588  0.0392157  0.0431373     0.0313726  0.0470588  0.0666667
 0.0431373  0.0431373  0.0509804     0.0392157  0.054902   0.0627451
 0.0431373  0.0431373  0.0431373     0.0313726  0.0509804  0.0705882
 0.0509804  0.0431373  0.0509804     0.0313726  0.0627451  0.0627451
 0.0470588  0.0431373  0.0431373  …  0.0235294  0.0470588  0.0745098
 0.0509804  0.0509804  0.0392157     0.0313726  0.0588235  0.0705882
 0.0470588  0.0470588  0.0431373     0.0313726  0.0588235  0.0941177
 ⋮                                ⋱                                 
 0.2

In [10]:
# convert the classes into labels, to train
yTrain = int(map(x -> x[2], labelsInfoTrain[:classname]))

 in depwarn at deprecated.jl:73
 in int at deprecated.jl:50
 in int at deprecated.jl:282
 in int at C:\Users\cyber\.julia\v0.4\DataArrays\src\dataarray.jl:619
 in include_string at loading.jl:282
 in execute_request_0x535c5df2 at C:\Users\cyber\.julia\v0.4\IJulia\src\execute_request.jl:182
 in eventloop at C:\Users\cyber\.julia\v0.4\IJulia\src\IJulia.jl:142
 in anonymous at task.jl:447
while loading In[10], in expression starting on line 2


22424-element Array{Int64,1}:
 48
 48
 48
 48
 48
 48
 48
 48
 48
 48
 48
 48
 48
  ⋮
 57
 57
 57
 57
 57
 57
 57
 57
 57
 57
 57
 57

In [11]:
size(xTrain)

(22424,19200)

In [14]:
# calculate the # of features as sqrt of the total number of features
nfeats = int(round((size(xTrain)[2]) ^ 0.5))

 in depwarn at deprecated.jl:73
 in int at deprecated.jl:50
 in include_string at loading.jl:282
 in execute_request_0x535c5df2 at C:\Users\cyber\.julia\v0.4\IJulia\src\execute_request.jl:182
 in eventloop at C:\Users\cyber\.julia\v0.4\IJulia\src\IJulia.jl:142
 in anonymous at task.jl:447
while loading In[14], in expression starting on line 2


139

In [15]:
# Train the random forest model
# number of features chosen at each random split = nfeats
# number of trees = 50
# subsampling ratio - 1.0
model = build_forest(yTrain, xTrain, nfeats, 50, 1.0)

Ensemble of Decision Trees
Trees:      50
Avg Leaves: 1368.1
Avg Depth:  25.7

In [16]:
save("./statefarm.jld", "model", model)

In [18]:
save("./xTrain.jld", "xTrain", xTrain)

In [17]:
save("./yTrain.jld", "yTrain", yTrain)